# Tool Creation Examples

From the docs 

"""
Invocation State Compared To Other Approaches

It's important to understand how invocation state compares to other approaches that impact tool execution:

    Tool Parameters: Use for data that the LLM should reason about and provide based on the user's request. Examples include search queries, file paths, calculation inputs, or any data the agent needs to determine from context.

    Invocation State: Use for context and configuration that should not appear in prompts but affects tool behavior. Best suited for parameters that can change between agent invocations. Examples include user IDs for personalization, session IDs, or user flags.

    Class-based tools: Use for configuration that doesn't change between requests and requires initialization. Examples include API keys, database connection strings, service endpoints, or shared resources that need setup.

""" [1] 

Bottom line up top:

Tool parameters can be adjusting by add to the @tool() decorator, adding inputs inside the brackets. 

Tool parameters use invocation_state to get structured inputs; e.g. `agent("Get my fake profile data. This is a test.", invocation_state={"user_id": "user123"})
`

Class based tools are tools that need a state that persists beyond a given execution, like a tool with a connection that needs to stay open. - there is nothing special about the class based connection with respect to the library so I am skipping the example breakout; this is more of are architecural approach using the standard __init__() to setup the class state, and decorating the class methods with @tool.

[1] https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/custom-tools

In [1]:
from strands import tool, Agent
from strands.models import BedrockModel
# the @tool decorator extracts information from your function's docstring to create the tool specification. 
#  The first paragraph becomes the tool's description, and the "Args" section provides parameter descriptions.


# Overriding Tool Name, Description, and Schema

In [4]:
@tool(name="get_weather", description="Retrieves weather forecast for a specified location")
def weather_forecast(city: str, days: int = 3) -> str:
    """Implementation function for weather forecasting.

    Args:
        city: The name of the city
        days: Number of days for the forecast
    """
    return f"Weather forecast for {city} for the next {days} days..."

In [5]:
@tool(
    inputSchema={
        "json": {
            "type": "object",
            "properties": {
                "shape": {
                    "type": "string",
                    "enum": ["circle", "rectangle"],
                    "description": "The shape type"
                },
                "radius": {"type": "number", "description": "Radius for circle"},
                "width": {"type": "number", "description": "Width for rectangle"},
                "height": {"type": "number", "description": "Height for rectangle"}
            },
            "required": ["shape"]
        }
    }
)
def calculate_area(shape: str, radius: float = None, width: float = None, height: float = None) -> float:
    """Calculate area of a shape."""
    if shape == "circle":
        return 3.14159 * radius ** 2
    elif shape == "rectangle":
        return width * height
    return 0.0


In [11]:
bedrock_model = BedrockModel(
    model_id="global.amazon.nova-2-lite-v1:0",  
    temperature=0.7,
    max_tokens=200,
)

agent = Agent(
    model=bedrock_model,
    tools=[calculate_area]
)


In [ ]:
# testing the radius tool 
result = agent("Calculate the area of a circle with radius 5")


Tool #1: calculate_area
The area of the circle with radius 5 is **78.54 square units** (rounded to two decimal places).

## Changing output

In [20]:
# testing the weather tool 

def some_other_function(source_id):
    return {'temperature': 5778, 'source': source_id}  

@tool
def sun_temperature_data(source_id: str) -> dict:
    """Fetch data from a specified source.

    Args:
        source_id: Identifier for the data source
    """
    try:
        data = some_other_function(source_id)
        return {
            "status": "success",
            "content": [ {
                "json": data,
            }]
        }
    except Exception as e:
        return {
            "status": "error",
             "content": [
                {"text": f"Error:{e}"}
            ]
        }

bedrock_model2 = BedrockModel(
    model_id="global.amazon.nova-2-lite-v1:0",  
    temperature=0.7,
    max_tokens=200,
)

agent2 = Agent(
    model=bedrock_model2,
    tools=[sun_temperature_data]
)


In [21]:
agent2("fetch the lateset data on sun temperature")


Tool #1: sun_temperature_data
The latest data on sun temperature indicates a temperature of **5778 K** (Kelvin). This value represents the effective temperature of the Sun's photosphere, which is the visible surface layer we observe from Earth. 

This temperature is derived from the latest available measurements from the designated data source (`sun_temperature_latest`).

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "The latest data on sun temperature indicates a temperature of **5778 K** (Kelvin). This value represents the effective temperature of the Sun's photosphere, which is the visible surface layer we observe from Earth. \n\nThis temperature is derived from the latest available measurements from the designated data source (`sun_temperature_latest`)."}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'sun_temperature_data': ToolMetrics(tool={'toolUseId': 'tooluse_F3reAqdbRnWfJd2ko02qYw', 'name': 'sun_temperature_data', 'input': {'source_id': 'sun_temperature_latest'}}, call_count=1, success_count=1, error_count=0, total_time=0.0006840229034423828)}, cycle_durations=[0.9228670597076416], agent_invocations=[AgentInvocation(cycles=[EventLoopCycleMetric(event_loop_cycle_id='8e935a6b-b297-4ab7-b7f8-0e61e7e31895', usage={'inputTokens': 928, 'outputTokens': 30, 'totalTokens': 958}), EventLoopCycleMetric(e

## Async Invocation¶

Strands Agents actually uses async calls by default without any asyncio implimentation. 

This is because the ConcurrentToolExecutor is the default tool executor. [1] 
Currently (today Jan 31st 2025) the documentation also indicates that you need special syntax but this is not the case. [2] 

We can use SequentialToolExecutor instead. 

== Ref ==

[1] https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/executors/
[2] https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/custom-tools/#async-invocation

In [9]:
import asyncio
import time
from strands import Agent, tool
from strands.models import BedrockModel
from strands.tools.executors import SequentialToolExecutor

@tool
async def call_api_sequential_1() -> str:

    start_time = time.time()
    print(f"Sequential API 1 call started at: {time.strftime('%H:%M:%S.%f', time.localtime(start_time))[:-3]}")
    
    await asyncio.sleep(3)  # 3 second async call
    
    end_time = time.time()
    print(f"Sequential API 1 call completed at: {time.strftime('%H:%M:%S.%f', time.localtime(end_time))[:-3]}")
    print(f"Sequential API 1 call duration: {end_time - start_time:.2f} seconds")
    
    return "Sequential API 1 result"

@tool
async def call_api_sequential_2() -> str:

    start_time = time.time()
    print(f"Sequential API 2 call started at: {time.strftime('%H:%M:%S.%f', time.localtime(start_time))[:-3]}")
    
    await asyncio.sleep(4)  # 4 second async call
    
    end_time = time.time()
    print(f"Sequential API 2 call completed at: {time.strftime('%H:%M:%S.%f', time.localtime(end_time))[:-3]}")
    print(f"Sequential API 2 call duration: {end_time - start_time:.2f} seconds")
    
    return "Sequential API 2 result"


In [11]:

start_time = time.time()
print(f"Sequential Agent invocation started at: {time.strftime('%H:%M:%S.%f', time.localtime(start_time))[:-3]}")

bedrock_model = BedrockModel(
    model_id="global.amazon.nova-2-lite-v1:0",  
    temperature=0.7,
    max_tokens=200,
)

agent = Agent(
    model=bedrock_model,
    tool_executor=SequentialToolExecutor(), 
    tools=[call_api_sequential_1, call_api_sequential_2],
    system_prompt="You are a helpful assistant that can call multiple APIs. When asked to call APIs, call both API 1 and API 2."
)

result = agent("Can you call both of my APIs?")

end_time = time.time()
print(f"Sequential Agent invocation completed at: {time.strftime('%H:%M:%S.%f', time.localtime(end_time))[:-3]}")
print(f"Sequential Total execution time: {end_time - start_time:.2f} seconds")

result

Sequential Agent invocation started at: 23:55:5

Tool #1: call_api_sequential_1

Tool #2: call_api_sequential_2
Sequential API 1 call started at: 23:55:5
Sequential API 1 call completed at: 23:56:0
Sequential API 1 call duration: 3.00 seconds
Sequential API 2 call started at: 23:56:0
Sequential API 2 call completed at: 23:56:0
Sequential API 2 call duration: 4.00 seconds
Both APIs have been successfully called. Here are the results:

1. Sequential API 1 returned: "Sequential API 1 result"
2. Sequential API 2 returned: "Sequential API 2 result" 

Is there anything specific you'd like to do with these results or any further actions you'd like me to take?Sequential Agent invocation completed at: 23:56:0
Sequential Total execution time: 8.68 seconds


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Both APIs have been successfully called. Here are the results:\n\n1. Sequential API 1 returned: "Sequential API 1 result"\n2. Sequential API 2 returned: "Sequential API 2 result" \n\nIs there anything specific you\'d like to do with these results or any further actions you\'d like me to take?'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'call_api_sequential_1': ToolMetrics(tool={'toolUseId': 'tooluse_F3EJH7VFQdGI7pF4nwUewQ', 'name': 'call_api_sequential_1', 'input': {}}, call_count=1, success_count=1, error_count=0, total_time=3.0033462047576904), 'call_api_sequential_2': ToolMetrics(tool={'toolUseId': 'tooluse_hUqKf0WbQ7WhOal_VzbLMA', 'name': 'call_api_sequential_2', 'input': {}}, call_count=1, success_count=1, error_count=0, total_time=4.002664089202881)}, cycle_durations=[0.7883310317993164], agent_invocations=[AgentInvocation(cycles=[EventLoopCycleMetric(event_loop_cycle_id='78479d



 Now lets we see the same thing but with the default and no special 
 implimentation still sees async execution. 

 


In [13]:


start_time = time.time()
print(f"Normal Agent invocation started at: {time.strftime('%H:%M:%S.%f', time.localtime(start_time))[:-3]}")

bedrock_model = BedrockModel(
    model_id="global.amazon.nova-2-lite-v1:0",  
    temperature=0.7,
    max_tokens=200,
)

agent = Agent(
    model=bedrock_model,
    tools=[call_api_sequential_1, call_api_sequential_2],
    system_prompt="You are a helpful assistant that can call multiple APIs. When asked to call APIs, call both API 1 and API 2."
)

result = agent("Can you call both of my APIs?")

end_time = time.time()
print(f"Normal Agent invocation completed at: {time.strftime('%H:%M:%S.%f', time.localtime(end_time))[:-3]}")
print(f"Normal Total execution time: {end_time - start_time:.2f} seconds")


Normal Agent invocation started at: 23:59:3

Tool #1: call_api_sequential_1

Tool #2: call_api_sequential_2
Sequential API 1 call started at: 23:59:3
Sequential API 2 call started at: 23:59:3
Sequential API 1 call completed at: 23:59:4
Sequential API 1 call duration: 3.00 seconds
Sequential API 2 call completed at: 23:59:4
Sequential API 2 call duration: 4.00 seconds
Both APIs have been successfully called:
- Sequential API 1 result
- Sequential API 2 result

Is there anything specific you'd like to know or do next with these results?Normal Agent invocation completed at: 23:59:4
Normal Total execution time: 5.56 seconds


# Tool Context 

After setting the context=True in the decorator and adding ToolContex (which is __) to the tool definition, you have access to:
- tool_use which has the name of the tool, a unique tool request id, and input parameters to the tool which can be any type.# 
- agent, which is the agent itself (and so you can reference its chat history and such )
- and invocation_state caller provided key word arguments passed to the agent

In [15]:
from strands import tool, Agent, ToolContext

@tool(context=True)
def get_self_name(tool_context: ToolContext) -> str:
    return f"The agent name is {tool_context.agent.name}"

@tool(context=True)
def get_tool_use_id(tool_context: ToolContext) -> str:
    return f"Tool use is {tool_context.tool_use["toolUseId"]}"

@tool(context=True)
def get_invocation_state(tool_context: ToolContext) -> str:
    return f"Invocation state: {tool_context.invocation_state["custom_data"]}"

agent = Agent(tools=[get_self_name, get_tool_use_id, get_invocation_state], name="Best agent")

bedrock_model = BedrockModel(
    model_id="global.amazon.nova-2-lite-v1:0",  
    temperature=0.7,
    max_tokens=200,
)

agent = Agent(
    model=bedrock_model,
    tools=[get_self_name, get_tool_use_id, get_invocation_state],
    name='Best Model'
)


agent("What is your name?")
agent("What is the tool use id?")
agent("What is the invocation state?", custom_data="You're the best agent ;)")


Tool #1: get_self_name
My name is **Best Model**. I'm here to assist you with any questions or tasks you might have!
Tool #2: get_tool_use_id
The tool use ID is **tooluse_et4FfRljQHmDTjvRV0fgUA**. 

If you need anything else or want to explore more functionality, just let me know!

/opt/anaconda3/lib/python3.12/site-packages/strands/agent/agent.py:413: UserWarning: `**kwargs` parameter is deprecating, use `invocation_state` instead.
  async for event in events:



Tool #3: get_invocation_state
The current invocation state is: **You're the best agent ;)** 

I'm glad to hear that! If there's anything specific you'd like to discuss or any assistance you need, I'm here to help.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "The current invocation state is: **You're the best agent ;)** \n\nI'm glad to hear that! If there's anything specific you'd like to discuss or any assistance you need, I'm here to help."}]}, metrics=EventLoopMetrics(cycle_count=6, tool_metrics={'get_self_name': ToolMetrics(tool={'toolUseId': 'tooluse_fWeyRBs8QXm_tgj-4sOCqQ', 'name': 'get_self_name', 'input': {}}, call_count=1, success_count=1, error_count=0, total_time=0.0006520748138427734), 'get_tool_use_id': ToolMetrics(tool={'toolUseId': 'tooluse_et4FfRljQHmDTjvRV0fgUA', 'name': 'get_tool_use_id', 'input': {}}, call_count=1, success_count=1, error_count=0, total_time=0.0004971027374267578), 'get_invocation_state': ToolMetrics(tool={'toolUseId': 'tooluse_7X076oOURb6J7dj-fGjy1Q', 'name': 'get_invocation_state', 'input': {}}, call_count=1, success_count=1, error_count=0, total_time=0.0007617473602294922)}, cycle_durations=[0.625762939453125, 0.6156

In [ ]:
### 'Invocation State' usage by passing key value pair in the chat

from strands import tool, Agent, ToolContext
import requests
import json

def fake_api(user_id):
    print('got user id: ', user_id)
    return {'fake_key': 'fake_value'}

@tool(context=True)
def api_call(query: str, tool_context: ToolContext) -> dict:
    """Make an API call with user context.

    Args:
        query: The search query to send to the API
        tool_context: Context containing user information
    """
    user_id = tool_context.invocation_state.get("user_id")

    response = fake_api(user_id)

    return response

bedrock_model = BedrockModel(
    model_id="global.amazon.nova-2-lite-v1:0",  
    temperature=0.7,
    max_tokens=200,
)

agent = Agent(
    model=bedrock_model,
    tools=[api_call]
    )

# the docs example at the time of writing (user_id="user123") is depracated
result = agent("Get my fake profile data. This is a test.", invocation_state={"user_id": "user123"})



Tool #1: api_call
got user id:  user123
Here's your fake profile data based on the test request:

```json
{
  "fake_key": "fake_value"
}
```

This appears to be a simple test response with placeholder data. If you need more detailed fake profile information or have a specific data structure in mind, please let me know and I can help generate more comprehensive test data for your needs.

## Module Based Usage 

by default files defining tools should go in the ./tools directory. 

You can also provide a direct path to the tool. 


In [ ]:
# this demonstrates loading a tool using the @tool decorator (hello_tool), and also one using TOOL_SPEC (weather_module)


bedrock_model = BedrockModel(
    model_id="global.amazon.nova-2-lite-v1:0",  
    temperature=0.7,
    max_tokens=200,
)

agent = Agent(
    model=bedrock_model,
    load_tools_from_directory=True
    )

print(agent.tool_names)

['weather_module', 'hello_test']


In [17]:
agent.tool.hello_test(name="Chris") 

{'toolUseId': 'tooluse_hello_test_540414264',
 'status': 'success',
 'content': [{'text': 'Hello, Chris!'}]}

In [18]:
agent.tool.weather_module(name="Chris") 

{'toolUseId': 'tooluse_weather_module_415779546',
 'status': 'success',
 'content': [{'text': 'Weather forecast for  for the next 3 days...'}]}

In [19]:

# we can also load tools providing the file path to the toos directly

bedrock_model = BedrockModel(
    model_id="global.amazon.nova-2-lite-v1:0",  
    temperature=0.7,
    max_tokens=200,
)

agent = Agent(
    model=bedrock_model,
    tools=['./tools/hello_tool.py']
    )

print(agent.tool_names)

['hello_test']


In [21]:
result = agent.tool.hello_test(name="Chris")
result

{'toolUseId': 'tooluse_hello_test_997023815',
 'status': 'success',
 'content': [{'text': 'Hello, Chris!'}]}